In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import types as T 
from pyspark.sql import functions as F 

In [2]:
spark = SparkSession.builder \
        .master('local[*]') \
        .appName('test') \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 17:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/01 17:14:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read \
           .parquet('./data/unified/green/*/*')

In [5]:
df_green.createOrReplaceTempView('trips_data')

In [28]:
df_result = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID as zone,    
    
    ROUND(SUM(total_amount), 2)  AS amount,
    COUNT(1) AS number_of_records
FROM 
    trips_data
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2; 
""")

In [29]:
df_result.show()

+-------------------+----+-------+-----------------+
|               hour|zone| amount|number_of_records|
+-------------------+----+-------+-----------------+
|2020-01-01 00:00:00|   7| 769.73|               45|
|2020-01-01 00:00:00|  17| 195.03|                9|
|2020-01-01 00:00:00|  18|    7.8|                1|
|2020-01-01 00:00:00|  22|   15.8|                1|
|2020-01-01 00:00:00|  24|   87.6|                3|
|2020-01-01 00:00:00|  25|  531.0|               26|
|2020-01-01 00:00:00|  29|   61.3|                1|
|2020-01-01 00:00:00|  32|  68.95|                2|
|2020-01-01 00:00:00|  33| 317.27|               11|
|2020-01-01 00:00:00|  35| 129.96|                5|
|2020-01-01 00:00:00|  36| 295.34|               11|
|2020-01-01 00:00:00|  37| 175.67|                6|
|2020-01-01 00:00:00|  38|  98.79|                2|
|2020-01-01 00:00:00|  40| 168.98|                8|
|2020-01-01 00:00:00|  41|1363.96|               84|
|2020-01-01 00:00:00|  42| 799.76|            

In [30]:
df_result.write.parquet('data/repoer/revenue/green')